In [1]:
import os

In [2]:
%pwd

'/home/himanshu/github_himanshu/end-to-end-ML-with-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/himanshu/github_himanshu/end-to-end-ML-with-MLflow'

In [5]:
from dataclasses import dataclass  # Import the dataclass decorator for creating data classes
from pathlib import Path  # Import the Path class for handling filesystem paths

@dataclass(frozen=True)  # Define the class as a dataclass and make it immutable with frozen=True
class DataTransformationConfig:
    """
    Configuration class for data transformation.

    Attributes:
        root_dir (Path): The root directory where data transformation operations will take place.
        data_path (Path): The path to the data that will be transformed.
    """
    root_dir: Path
    data_path: Path


In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        """
        Initialize ConfigurationManager with paths to the configuration files.
        
        Args:
            config_filepath (Path): Path to the configuration YAML file.
            params_filepath (Path): Path to the parameters YAML file.
            schema_filepath (Path): Path to the schema YAML file.
        """
        self.config = read_yaml(config_filepath)  # Read the configuration file
        self.params = read_yaml(params_filepath)  # Read the parameters file
        self.schema = read_yaml(schema_filepath)  # Read the schema file

        create_directories([self.config.artifacts_root])  # Create the root directory for artifacts

    def get_data_transformation_config(self) -> DataTransformationConfig:
        """
        Get the DataTransformationConfig object from the configuration.
        
        Returns:
            DataTransformationConfig: Configuration for data transformation.
        """
        config = self.config.data_transformation  # Get the data transformation configuration section

        create_directories([config.root_dir])  # Ensure the root directory for data transformation exists

        # Create and return a DataTransformationConfig object with the configuration details
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config


In [8]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd 

[2024-06-13 13:27:31,519: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-06-13 13:27:31,519: INFO: utils: NumExpr defaulting to 8 threads.]


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        """
        Initialize the DataTransformation class with a configuration.
        
        Args:
            config (DataTransformationConfig): Configuration for data transformation.
        """
        self.config = config  # Store the configuration

    def train_test_spliting(self):
        """
        Split the data into training and test sets and save them to files.
        """
        try:
            # Load the data from the specified data path
            data = pd.read_csv(self.config.data_path)

            # Split the data into training and test sets with a 75-25 split
            train, test = train_test_split(data, test_size=0.25, random_state=42)

            # Save the training data to a CSV file in the root directory
            train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
            
            # Save the test data to a CSV file in the root directory
            test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

            # Log the success of the data split
            logger.info("Split data into training and test sets")
            logger.info(f"Training data shape: {train.shape}")
            logger.info(f"Test data shape: {test.shape}")

            # Print the shapes of the training and test sets
            print(f"Training data shape: {train.shape}")
            print(f"Test data shape: {test.shape}")

        except Exception as e:
            logger.error(f"Error during train-test split: {e}")
            raise e


In [10]:
try:
    # Initialize ConfigurationManager with default file paths
    config = ConfigurationManager()
    
    # Retrieve data transformation configuration
    data_transformation_config = config.get_data_transformation_config()
    
    # Initialize DataTransformation with the retrieved configuration
    data_transformation = DataTransformation(config=data_transformation_config)
    
    # Perform train-test splitting
    data_transformation.train_test_spliting()
except Exception as e:
    # Raise the exception if any error occurs
    raise e


[2024-06-13 13:27:31,768: INFO: common: YAML file: config/config.yaml loaded successfully]
[2024-06-13 13:27:31,769: INFO: common: YAML file: params.yaml loaded successfully]
[2024-06-13 13:27:31,772: INFO: common: YAML file: schema.yaml loaded successfully]
[2024-06-13 13:27:31,773: INFO: common: Created directory at: artifacts]
[2024-06-13 13:27:31,774: INFO: common: Created directory at: artifacts/data_transformation]
[2024-06-13 13:27:31,796: INFO: 1022534127: Split data into training and test sets]
[2024-06-13 13:27:31,797: INFO: 1022534127: Training data shape: (1199, 12)]
[2024-06-13 13:27:31,797: INFO: 1022534127: Test data shape: (400, 12)]
Training data shape: (1199, 12)
Test data shape: (400, 12)
